In [1]:
# !pip install transformers -q
# !pip install datasets -q
# !pip install evaluate -q

In [2]:
import os
os.environ["WANDB_DISABLED"] = "true"
import evaluate
import pandas as pd

df_train = pd.read_csv('/home/aflah20082/ML_Project/Data/AugmentedData/english_train_augmented.csv')
df_test = pd.read_csv('/home/aflah20082/ML_Project/Data/PreprocessedData/english_test_preprocess.csv')
df_val = pd.read_csv('/home/aflah20082/ML_Project/Data/PreprocessedData/english_dev_preprocess.csv')

In [3]:
label_replacement = {
    'Hope_speech': 0,
    'Non_hope_speech': 1,
    'not-English': 2,
}

df_train['label'] = df_train['label'].replace(label_replacement)
df_test['label'] = df_test['label'].replace(label_replacement)
df_val['label'] = df_val['label'].replace(label_replacement)

# Drop rows with label not-English
df_train = df_train[df_train['label'] != 2]
df_test = df_test[df_test['label'] != 2]
df_val = df_val[df_val['label'] != 2]

In [4]:
df_train = df_train[['augmented_text', 'label']]
df_test = df_test[['preprocessed_text', 'label']]
df_val = df_val[['preprocessed_text', 'label']]

In [5]:
df_train = df_train.rename(columns={'augmented_text': 'text'})
df_test = df_test.rename(columns={'preprocessed_text': 'text'})
df_val = df_val.rename(columns={'preprocessed_text': 'text'})

In [6]:
from datasets import Dataset, DatasetDict
tds = Dataset.from_pandas(df_train[['text', 'label']], preserve_index=False)
vds = Dataset.from_pandas(df_val[['text', 'label']], preserve_index=False)
testds = Dataset.from_pandas(df_test[['text', 'label']], preserve_index=False)


ds = DatasetDict()

ds['train'] = tds
ds['validation'] = vds
ds['test'] = testds

In [7]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base")

# tokenizer.add_special_tokens(["<intent>", "</intent>"])

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = ds.map(tokenize_function, batched=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


  0%|          | 0/43 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [8]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42)
small_eval_dataset = tokenized_datasets["validation"].shuffle(seed=42)

In [9]:
import numpy as np
metric1 = evaluate.load("accuracy")
metric2 = evaluate.load("precision")
metric3 = evaluate.load("recall")
metric4 = evaluate.load("f1")

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [10]:
from transformers import TrainingArguments, Trainer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("vinai/bertweet-base", num_labels=2) 

training_args = TrainingArguments(output_dir=f"Model Dumps/bertweet_data_augmented_finetune", evaluation_strategy="epoch")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)
trainer.train()

Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-base and are newly initialized: 

Epoch,Training Loss,Validation Loss,Accuracy
1,0.237900,0.273189,0.923266
2,0.130500,0.310784,0.924674
3,0.060700,0.394244,0.925730


Saving model checkpoint to Model Dumps/bertweet_data_augmented_finetune/checkpoint-500
Configuration saved in Model Dumps/bertweet_data_augmented_finetune/checkpoint-500/config.json
Model weights saved in Model Dumps/bertweet_data_augmented_finetune/checkpoint-500/pytorch_model.bin
Saving model checkpoint to Model Dumps/bertweet_data_augmented_finetune/checkpoint-1000
Configuration saved in Model Dumps/bertweet_data_augmented_finetune/checkpoint-1000/config.json
Model weights saved in Model Dumps/bertweet_data_augmented_finetune/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to Model Dumps/bertweet_data_augmented_finetune/checkpoint-1500
Configuration saved in Model Dumps/bertweet_data_augmented_finetune/checkpoint-1500/config.json
Model weights saved in Model Dumps/bertweet_data_augmented_finetune/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to Model Dumps/bertweet_data_augmented_finetune/checkpoint-2000
Configuration saved in Model Dumps/bertweet_data_augmente

TrainOutput(global_step=15885, training_loss=0.16217347188006417, metrics={'train_runtime': 5511.2717, 'train_samples_per_second': 23.058, 'train_steps_per_second': 2.882, 'total_flos': 8359038228787200.0, 'train_loss': 0.16217347188006417, 'epoch': 3.0})

In [11]:
predictions = trainer.predict(tokenized_datasets["validation"])
preds = np.argmax(predictions.predictions, axis=-1)
accuracy = metric1.compute(predictions=preds, references=predictions.label_ids)
precision = metric2.compute(predictions=preds, references=predictions.label_ids)
recall = metric3.compute(predictions=preds, references=predictions.label_ids)

The following columns in the test set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text.
***** Running Prediction *****
  Num examples = 2841
  Batch size = 8


In [12]:
print("Accuracy: ", accuracy)
print("Precision: ", precision)
print("Recall: ", recall)

Accuracy:  {'accuracy': 0.9257303766279479}
Precision:  {'precision': 0.9422355588897224}
Recall:  {'recall': 0.9778123783573375}


In [13]:
# Predictions on test set
predictions = trainer.predict(tokenized_datasets["test"])
preds = np.argmax(predictions.predictions, axis=-1)
accuracy = metric1.compute(predictions=preds, references=predictions.label_ids)
precision = metric2.compute(predictions=preds, references=predictions.label_ids)
recall = metric3.compute(predictions=preds, references=predictions.label_ids)
macro_f1 = metric4.compute(predictions=preds, references=predictions.label_ids, average='macro')
print("Accuracy: ", accuracy)
print("Precision: ", precision)
print("Recall: ", recall)
print("Macro F1: ", macro_f1)

The following columns in the test set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text.
***** Running Prediction *****
  Num examples = 2843
  Batch size = 8


Accuracy:  {'accuracy': 0.9194512838550827}
Precision:  {'precision': 0.9440270473328325}
Recall:  {'recall': 0.9691477053605861}
Macro F1:  {'f1': 0.7125499745022419}
